# testing pyhmmer work

In [11]:
# system dependecies
import logging
import os
import sys
import tempfile
import time
from typing import Union

# library dependencies
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import duckdb as ddb
from joblib import Parallel, delayed
import pandas as pd
import pyhmmer

# local dependencies
import learn2therm.utils

In [12]:
PATH = os.path.abspath("/Users/humoodalanzi/pfam/proteins copy/*.parquet")
HMM_PATH = '/Users/humoodalanzi/pfam/Pfam-A.hmm'  # ./Pfam-A.hmm

## testing ground for loading funciton

In [3]:
# stolen from Evan's t1.0 scripts
with tempfile.TemporaryDirectory(dir='./') as tmpdir:
    # Establishing a connection with Duck DB
    conn = ddb.connect(tmpdir+'proteins.db', read_only=False)
    # Making a SQL table
    conn.execute("CREATE TABLE proteins AS SELECT * FROM read_parquet('../data/uniprot_chunk_0.parquet')")
    # Committing DB
    conn.commit()

     # get some test proteins to run resource test alignment on
    # considering the max protein length
    query_proteins = conn.execute(
        f"""SELECT pid, protein_seq, LENGTH(protein_seq) AS len FROM proteins 
        WHERE len<=250
        ORDER BY RANDOM()
        LIMIT 1000
        """).df()
    subject_proteins = conn.execute(
        f"""SELECT pid, protein_seq, LENGTH(protein_seq) AS len FROM proteins 
        WHERE len<=250
        ORDER BY RANDOM()
        LIMIT 1000
        """).df()

    # get some metadata about taxa protein join
    # protein total counts per organism was tracked in s0.3
    # but lets recompute that data considering a max protien length
    protein_counts = conn.execute(
        f"""SELECT taxid, COUNT(*) AS n_proteins
        FROM proteins
        WHERE LENGTH(protein_seq)<=250
        GROUP BY taxid""").df()

In [4]:
query_proteins

,pid,protein_seq,len
0,I3R7V8,MPSEKEKMLAGELYDASDPELVMERKRARRLTRRFNASDETESLRR...,185
1,L0JM53,MIGSDVVVAGSALAIAVAVGLVAHEWSHAAVLRLARVEYAVSYFPG...,149
2,I3R9Q8,MYEHILVPTDGSETAEYAVDQAVDIASKYGSTVHALYVIDVDATSY...,154
3,L0JY84,MKDVKLDPSEESTYECFDCGTVVVSATAPGRCPSCGADMRNRATPLE,47
4,I3R1G7,MANGKYAARKLKKDRQKRRWSDSEYARRERGLGKKSDPLEGAPQGR...,142
...,...,...,...
995,A0A1L3Q032,MKYVKEFVKLAKSDLKSSVVLYENKCYPQSIFFFAQSVEKANKALA...,193
996,D2RQ33,MRKSGTWMTIWDDRILETLRKEGGKPVGELAEEDGIRISHSSVSRR...,103
997,A0A0A7GGQ8,MPLFLTRKAIEEVDVGDVIEVHADDPSARKDIPEWAERAGHKVLSV...,61
998,A0A650CYC0,MPTIHLSIPEGMYEELRKKAEDMGIQITDLVKFYIRQGMEKEEEGE...,101


In [5]:
subject_proteins

,pid,protein_seq,len
0,A0A1I0FQE4,MDVTQIGLGATLLVIGTLTLIGPATLATGPLAYLVTGSTLVVTVAA...,58
1,A0A1I0M298,MGFGSYDESEQQEQTTSDEDVEAVNVHENDHEGKLSFESDLSTDEL...,60
2,A0A0Q2QPZ4,MYKIKDEWGEFLVRLARRAIEEYVRNGRTIKPPEDTPPQLWERMGV...,204
3,A0A0F8FH29,MTRFIKYHPRSNTYVIEKRAFLEEDLTLDGNVIVGPEVKFWKNLTV...,142
4,B8D531,MNRVYVTVLATLLIIVGLLLGASMWLDILRANTYVDTGELDWEIVE...,225
...,...,...,...
995,A0A1H9YYW1,MNNIEEMIQKAVELQANGLVTGQIANELNVSRETVTWLLTRSKKDV...,203
996,L0JKK2,MGTHVLVPLDGSSQAWAAFDHAVSNHDGGRITTLHVVDPMAGVYSD...,146
997,L0JZ77,MCVSSRMPITVVQLTLSALAAVFAFITAVYGRLNYADGELDRQKKI...,195
998,L0JZQ8,MTVFALLRVTPITDDDATEDVAAAIDALEEYDVEYETTPLATTLEA...,104


So, in theory, I could sample the .parquet files and create a df out of them for loading

In [ ]:
# goal to make this function
def load_protein_data(database, **kwargs):
    """
    Load the protein sequences and associated taxonomic information from the input CSV files
    for DB version 1.0, or from the input Parquet files for DB version 1.1.

    Parameters
    ----------
    database : _type_
        _description_
    """
    # TODO
    pass

## Putting the pieces

In [3]:
# stolen from Evan's t1.0 scripts
with tempfile.TemporaryDirectory(dir='../tmp/') as tmpdir:
    # Establishing a connection with Duck DB
    conn = ddb.connect(tmpdir+'proteins.db', read_only=False)
    # Making a SQL table
    conn.execute(f"CREATE TABLE proteins AS SELECT * FROM read_parquet('../data/uniprot_chunk_0.parquet')")
    # Committing DB
    conn.commit()

    # get some test proteins to run resource test alignment on
    # considering the max protein length
    query_proteins = conn.execute(
        """SELECT pid, protein_seq, LENGTH(protein_seq) AS len FROM proteins 
        WHERE len<=250
        ORDER BY RANDOM()
        LIMIT 1000
        """).df()

    # get some metadata about taxa protein join
    # protein total counts per organism was tracked in s0.3
    # but lets recompute that data considering a max protien length
    protein_counts = conn.execute(
        """SELECT taxid, COUNT(*) AS n_proteins
        FROM proteins
        WHERE LENGTH(protein_seq)<=250
        GROUP BY taxid""").df()

In [10]:
query_proteins

,pid,protein_seq,len
0,M0ATD1,MSKISRFTGKVVTLAKNAVGDRGESAAPQGGSGFADYAVVSLHCLR...,219
1,A0A238UTT8,MPLLQFDTTLTLSPSDRRAFAERVTDVYTDEMATERGHVAVTIRER...,123
2,A0A0A7GJ53,MLVTTGVCKSFGNHEVLKGIDFRVAPGEFAAIFAPSGSGKTTLLNI...,220
3,A0A0K1ISF3,MTETWDSAGYIASSRYRLAVCRYLSEHGSGLPSRIAAETDLAQPHV...,225
4,A0A830FPK4,MANLFGASLSLVLVVAGVALCIAEAFAPGAHFVVIGVALLAAGLAG...,190
...,...,...,...
995,E4NUJ6,MGKSSTSPSRRGGKGPKIERVAERYSITGLGDELVERWRGDQGEQE...,206
996,I3R8N2,MTVRLRTVTESDLPVIRTLYEPFVEETAITFAYDPPSVTDLESKLE...,201
997,D3SUA1,METVAVLNAIVDNLIEMNEYFSGVATGEGAAPLLMIAGTLLVVFSL...,64
998,O59571,MYKIVTVKDVVRIPPRMFTMDPKEAAMLVLRDTYEGTYDRDEGVIL...,188


The lesson here is that I shouldn't do maby parquets w/o Hyak. Need to figure out to use vscode with Hyak.

In [6]:
def prefetch_targets(hmms_path: str):
    """
    Prefetch HMM profiles from a given HMM database.
    Parameters
    ----------
    hmms_path : str
        Path to the HMM database.
    Returns
    -------
    targets : pyhmmer.plan7.OptimizedProfileBlock
        The HMM profiles loaded from the database.
    """
    # amino acid alphabet and prefetched inputs
    amino_acids = pyhmmer.easel.Alphabet.amino()
    optimized_profiles = list(pyhmmer.plan7.HMMPressedFile(hmms_path))
    targets = pyhmmer.plan7.OptimizedProfileBlock(
        amino_acids, optimized_profiles)
    return targets

In [7]:
def save_sequences_to_fasta(
        sequences: pd.core.frame.DataFrame,
        inputname: str = "input"):
    """
    Returns a list of SeqRecord objects and creates a corresponding input Fasta of them
    Parameters:
    ------------
    sequences : pandas.core.frame.DataFrame
        a dataframe with string amino acid sequences in a 'protein_seq' column
    input name : str, default = 'input'
        a name for the input fasta file
    Returns:
    ------------
    file : TextIOWrapper
        the input fasta file created from the list of SeqRecord objects
    Raises
    -------
    ValueError :
        if the input dataframe is empty
    AttributeError :
        if any of the sequences are invalid
    """
    # ensure input file has .fasta extension
    if not inputname.endswith('.fasta'):
        inputname = f"{os.path.splitext(inputname)[0]}.fasta"

    # check if input is empty
    if sequences.empty:
        raise ValueError("Input dataframe is empty")

    # check if sequences are valid
    for seq in sequences['protein_seq']:
        try:
            Seq(seq)
        except BaseException as exc:
            raise AttributeError("Invalid sequence") from exc

    # function
    records = []
    for index, seq in sequences.itertuples():
        try:
            record = SeqRecord(Seq(seq), id=str(index))
            records.append(record)
        except AttributeError as exc:
            raise AttributeError(f"Invalid sequence: {seq}") from exc

    # raise error if seq not valid
    if not records:
        raise AttributeError("No valid sequences found in input")

    with open(inputname, "w", encoding="utf-8") as file:
        SeqIO.write(records, file, "fasta")
    return file

In [8]:
def run_pyhmmer(
        input_file: str,
        hmms_path: str,
        prefetch: bool = False,
        output_file: str = None,
        cpu: int = 4,
        eval_con: float = 1e-10):
    """
    Run HMMER's hmmscan program on a set of input sequences using with HMMs from a database.
    Parameters
    ----------
    input_file : str
        Path to the input sequence file.
    hmms_path : str
        Path to the HMM database.
    prefetch : bool, optional
        Specifies how the HMM are stored in meomry.
    output_file : str, optional
        Path to the output file if the users wants to write the file.
    cpu : int, optional
        The number of CPUs to use. Default is 4.
    eval_con : float, optional
        E-value threshold for domain reporting. Default is 1e-10.
    Returns
    -------
    all_hits : pyhmmer.plan7.TopHits or domtblout file
        If the output_file has a name, it will be written to a domtblout file.
        Otherwise, the user will get a list of pyhmmeer TopHits objects.
    Notes
    -----
    This function runs HMMER's hmmscan program on a set of input sequences
    using HMMs from a given database.
    The function supports two modes: normal mode and prefetching mode.
    In normal mode, the HMMs are pressed and stored in a directory before execution.
    In prefetching mode, the HMMs are kept in memory for faster search.
    """
    # ensure input file has .fasta extension
    if not input_file.endswith('.fasta'):
        input_file = f"{os.path.splitext(input_file)[0]}.fasta"

    # ensure output_file has .domtblout extension
    if not output_file.endswith('.domtblout'):
        output_file = f"{os.path.splitext(output_file)[0]}.domtblout"

    # HMM profile modes
    if prefetch:
        targets = prefetch_targets(hmms_path)
    else:
        targets = pyhmmer.plan7.HMMFile("../data/pfam/.h3m")

    # HMMscan execution with or without saving output to file
    with pyhmmer.easel.SequenceFile(input_file, digital=True) as seqs:
        all_hits = list(pyhmmer.hmmer.hmmscan(seqs, targets, cpus=cpu, E=eval_con))
        # check if we should save the output
        if output_file is not None:
            with open(output_file, "wb") as dst:
                for i, hits in enumerate(all_hits):
                    hits.write(dst, format="domains", header=i == 0)

    return all_hits

In [9]:
def parse_pyhmmer(all_hits):
    """
    Parses the TopHit pyhmmer object getting the query and accession IDs and saves to a DataFrame
    Parameters
    ----------
    all_hits : list
        A list of TopHit objects from pyhmmer.
    Returns
    -------
    pandas.DataFrame
        A dataframe containing the query and accession IDs.
    """
    # initialize an empty dictionary to store the data
    parsed_hits = {}

    # iterate over each protein hit
    for top_hits in all_hits:
        for hit in top_hits:
            # extract the query and accession IDs and decode the query ID
            query_id = hit.hits.query_name.decode('utf-8')
            accession_id = hit.accession.decode('utf-8')

            # if the query_id already exists in the dictionary, append the accession_id
            # to the existing value
            if query_id in parsed_hits:
                parsed_hits[query_id].append(accession_id)
            # otherwise, create a new key-value pair in the dictionary
            else:
                parsed_hits[query_id] = [accession_id]

    # create the DataFrame from the dictionary and convert list of accession IDs to string
    df = pd.DataFrame(parsed_hits.items(), columns=["query_id", "accession_id"])
    df["accession_id"] = df["accession_id"].apply(lambda x: ';'.join(x))

    return df

In [ ]:
def worker_function(chunk_index, sequences, which, wakeup=None):
    """
    A wrapping function that runs and parses pyhmmer in chunks
    Parameters
    ----------
    chunk_index : int
        number of sequences chunks
    sequences : str
        a list of dataframe containing protein sequences
    """
    # we want to wait for execution to see if this worker is actually being used
    # or if it is in the process of being killed
    if wakeup is not None:
        time.sleep(wakeup)
    
    # define paths for input and output files
    input_file_path = f"./results/{which}_input_{chunk_index}"
    output_file_path = f"./results/{which}_output_{chunk_index}"

    # convert sequences to FASTA files
    save_sequences_to_fasta(sequences, input_file_path)

    # run HMMER via pyhmmer
    hits = run_pyhmmer(
        input_file=input_file_path,
        hmms_path=PFAM_PATH,
        prefetch=True,
        output_file=output_file_path,
        cpu=1,
        eval_con=1e-5)

    # Parse pyhmmer output and save to CSV file
    accessions_parsed = parse_pyhmmer(all_hits=hits)
    accessions_parsed.to_csv(
        f"./results/{which}_result_{chunk_index}.csv",
        index=False)